In [27]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [28]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


68.0 x 143 x 19.0


In [ ]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

In [36]:
cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]", 'ok'])

In [32]:
if "Alarm" in cnc.status:
    cnc.home()

Exception: []

In [7]:
cnc.init()

True

# Generate Program


In [8]:
grid_spacing = 2.5 # [mm]

In [9]:
Xs = np.arange(grid_spacing, BlockLength, grid_spacing)
Ys = np.arange(grid_spacing, BlockHeight, grid_spacing)

In [10]:
Xs

array([  2.5,   5. ,   7.5,  10. ,  12.5,  15. ,  17.5,  20. ,  22.5,
        25. ,  27.5,  30. ,  32.5,  35. ,  37.5,  40. ,  42.5,  45. ,
        47.5,  50. ,  52.5,  55. ,  57.5,  60. ,  62.5,  65. ,  67.5,
        70. ,  72.5,  75. ,  77.5,  80. ,  82.5,  85. ,  87.5,  90. ,
        92.5,  95. ,  97.5, 100. , 102.5, 105. , 107.5, 110. , 112.5,
       115. , 117.5, 120. , 122.5, 125. , 127.5, 130. , 132.5, 135. ,
       137.5, 140. , 142.5])

In [11]:
Ys

array([ 2.5,  5. ,  7.5, 10. , 12.5, 15. , 17.5, 20. , 22.5, 25. , 27.5,
       30. , 32.5, 35. , 37.5, 40. , 42.5, 45. , 47.5, 50. , 52.5, 55. ,
       57.5, 60. , 62.5, 65. , 67.5])

In [12]:
len(Xs)

57

In [13]:
Zs=[-5, 0, 5]+-14
Zs

TypeError: can only concatenate list (not "int") to list

In [14]:
Zs=np.array([-5, 0, 5])+-14
Zs

array([-19, -14,  -9])

In [15]:
n_powers = int(len(Xs)/len(Zs))
powers = np.linspace(50, 255, n_powers, dtype=np.uint8)
powers

array([ 50,  61,  72,  84,  95, 106, 118, 129, 141, 152, 163, 175, 186,
       198, 209, 220, 232, 243, 255], dtype=uint8)

In [16]:
dwells = [0.05, 0.10, 0.50, 1, 2, 5, 10]

In [17]:
len(dwells)

7

In [18]:
for idx3, dwell in enumerate(dwells):
    for idx1, Z in enumerate(Zs):
        for idx2, power in enumerate(powers):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3]     

In [19]:
def init(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=500)
    program.G21() # Metric Units
    program.G90() # Absolute positioning.
    return program

In [20]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = init(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

In [21]:
dot()

<GCode>[cmds=9]

In [22]:
programs=list()
for idx3, dwell in enumerate(dwells):
    for idx1, Z in enumerate(Zs):
        for idx2, power in enumerate(powers):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3]
            cfg = {
                "X": X,
                "Y": Y,
                "Z": Z,
                "power": power,
                "dwell": dwell,
                "machine": cnc
            }
            program = dot(**cfg)
            programs.append(program)

In [23]:
programs

[<GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[cmds=9],
 <GCode>[c

In [24]:
import time

In [25]:
for idx, program in enumerate(programs):
    print("{}: ".format(idx), end="")
    t1=time.time()
    out = program.run()
    t2=time.time()
    print("{}/{}".format(out, t2-t2))

0: None/0.0
1: None/0.0
2: None/0.0
3: None/0.0
4: None/0.0
5: None/0.0
6: None/0.0
7: None/0.0
8: None/0.0
9: None/0.0
10: None/0.0
11: None/0.0
12: None/0.0
13: None/0.0
14: None/0.0
15: None/0.0
16: None/0.0
17: None/0.0
18: None/0.0
19: None/0.0
20: None/0.0
21: None/0.0
22: None/0.0
23: None/0.0
24: None/0.0
25: None/0.0
26: None/0.0
27: None/0.0
28: None/0.0
29: None/0.0
30: None/0.0
31: None/0.0
32: None/0.0
33: None/0.0
34: None/0.0
35: None/0.0
36: None/0.0
37: None/0.0
38: None/0.0
39: None/0.0
40: None/0.0
41: None/0.0
42: None/0.0
43: None/0.0
44: None/0.0
45: None/0.0
46: None/0.0
47: None/0.0
48: None/0.0
49: None/0.0
50: None/0.0
51: None/0.0
52: None/0.0
53: None/0.0
54: None/0.0
55: None/0.0
56: None/0.0
57: None/0.0
58: None/0.0
59: None/0.0
60: None/0.0
61: None/0.0
62: None/0.0
63: None/0.0
64: None/0.0
65: None/0.0
66: None/0.0
67: None/0.0
68: None/0.0
69: None/0.0
70: None/0.0
71: None/0.0
72: None/0.0
73: None/0.0
74: None/0.0
75: None/0.0
76: None/0.0
77: None/

In [ ]:
programs[0]